# Sugar Tester

The aim of this notebook is to provide simple tests and example code for Sugar mais classes and features :
* Trainer / SGDTrainer
* Solver / SGDSolver
* NetworkEvaluator (no current evaluator implementations)

These code snippets have the same purpose than test functions. They are dedicated to test features difficult to test without real data or are complementary tests for already tested features.

**Hence, this notebook cells should be executed after any modifications of Sugar core features**

**Users should add code test snippets for the classes heritated from Trainer / Solver / NetworkEvaluator they implement**

In [ ]:
import sys
sys.path.append('/home/corentin/software/caffe-segnet-cudnn7/python/')
from pathlib import Path

from datum.datasets import Dataset
from datum.readers import SegmentationDatasetReader

camVid2_labels = ['Sky', 'Other', 'Unlabelled']
colors = {'Sky': [128,128,128], 'Building': [128,0,0], 'Pole': [192,192,128],
          'Road_marking': [255,69,0], 'Road': [128,64,128], 'Pavement': [60,40,222],
          'Tree': [128,128,0], 'SignSymbol': [192,128,128], 'Fence': [64,64,128],
          'Vehicle': [64,0,128], 'Pedestrian': [64,64,0], 'Bicyclist': [0,128,192],
          'Unlabelled': [0,0,0], 'Other': [255,255,255]}

## Test SGD solver

In [ ]:
from sugar.solvers import SGDSolver

solver_prototxt = '/home/corentin/tmp/test_solver.prototxt'
solver = SGDSolver.from_caffe_solver_protoxt(solver_prototxt)
print(solver.to_dict())

## Test Sugar SGD Trainer

### Test trainer with automatic data load

In [ ]:
import caffe
from datum.feeders.segnet_feeder import SegnetFeeder
from sugar.trainers import SGDTrainer
from sugar.solvers import SGDSolver

# 1\ Construct datasets
# Load CamVid test segmentation dataset
images_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/test/')
segmaps_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/testannot/')
reader = SegmentationDatasetReader('png', 'png',
                                   images_dir=images_dir,
                                   segmaps_dir=segmaps_dir)
test_dataset = Dataset()
reader.feed(test_dataset)

# Load CamVid train segmentation dataset
images_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/train/')
segmaps_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/trainannot/')
reader = SegmentationDatasetReader('png', 'png',
                                   images_dir=images_dir,
                                   segmaps_dir=segmaps_dir)
train_dataset = Dataset()
reader.feed(train_dataset)

# 2\ Construct solver, trainer and train
root_dir = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm/')
train_net_prototxt = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm/train.prototxt')
solver_prototxt = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm/solver.prototxt')
solver = SGDSolver.from_caffe_solver_protoxt(solver_prototxt)

trainer = SGDTrainer(root_dir,
                     train_net_prototxt, ['Convolution'],
                     train_input_loading_mode = 'AUTOMATIC',
                     solver=solver,
                     display_interval=5,
                     eval_interval=10,
                     snapshot_interval=20)
results = trainer.train()

### Test trainer with custom data load

In [ ]:
import caffe
from datum.feeders.segnet_feeder import SegnetFeeder
from sugar.trainers import SGDTrainer
from sugar.solvers import SGDSolver

# 1\ Construct datasets
# Load CamVid test segmentation dataset
images_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/test/')
segmaps_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/testannot/')
reader = SegmentationDatasetReader('png', 'png',
                                   images_dir=images_dir,
                                   segmaps_dir=segmaps_dir)
test_dataset = Dataset()
reader.feed(test_dataset)

# Load CamVid train segmentation dataset
images_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/train/')
segmaps_dir = Path('/home/corentin/software/SegNet-Tutorial/CamVid_352x480_gs_1class/trainannot/')
reader = SegmentationDatasetReader('png', 'png',
                                   images_dir=images_dir,
                                   segmaps_dir=segmaps_dir)
train_dataset = Dataset()
reader.feed(train_dataset)

# 2\ Construct Segnet Feeder
net_input_type = 'GRAY'
net_input_dim = (12, 1, 352, 480)
preprocessing_mode = 'no_preproc'
mean_channel = None
segnet_train_feeder = SegnetFeeder(dataset = train_dataset,
                                   net_input_type = net_input_type,
                                   net_input_dim = net_input_dim,
                                   preprocessing_mode = preprocessing_mode,
                                   mean_channel = mean_channel,
                                   gt_segmap_attribute = 'segmap_path',
                                   shuffle = True, seed = 42,
                                   force_batch_size=True)

# 3\ Construct data feeder, solver, trainer and train
root_dir = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm_customdataload/')
train_net_prototxt = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm_customdataload/train.prototxt')
solver_prototxt = Path('/home/corentin/experiments/segmentation/trainings/020420/custom_segnet_basic_352x480_nolrn_gs_no_batchnorm_customdataload/solver.prototxt')
solver = SGDSolver.from_caffe_solver_protoxt(solver_prototxt)

trainer = Trainer(root_dir,
                  train_net_prototxt, ['Convolution'],
                  train_input_loading_mode = 'CUSTOM',
                  train_feeder=segnet_train_feeder,
                  solver=solver,
                  display_interval=5,
                  eval_interval=10,
                  snapshot_interval=20)
results = trainer.train()